In [19]:
import pandas as pd
import numpy as np
import networkx as nx
import tqdm
import pickle

import pickle
import argparse
import os
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import node2vec
from gensim.models import Word2Vec
from sklearn import metrics, model_selection, pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [20]:
body = pd.read_csv('./input/soc-redditHyperlinks-body.tsv', delimiter = '\t')
title = pd.read_csv('./input/soc-redditHyperlinks-title.tsv', delimiter = '\t')

combined = pd.concat([title, body], axis = 0).reset_index(drop = True)
source = combined.SOURCE_SUBREDDIT.unique()
target = combined.TARGET_SUBREDDIT.unique()

In [21]:
a = list(set(combined['SOURCE_SUBREDDIT'].values.tolist() + combined['TARGET_SUBREDDIT'].values.tolist()))

In [22]:
a

['gamingposters',
 'paranormalhelpline',
 'suboxone',
 'cutemodeslutmode',
 'wargaming',
 'counting_gold',
 'kitchens',
 'madden15fantasydraft',
 'gtacartel',
 'mami',
 'henna',
 'u3pexchange',
 'galaxynote4',
 'peoplelinkingtosubs',
 'djibouti',
 'cannotwatchscottstots',
 'streetfighterex',
 'fivenightsatfanfic',
 'fifaballondor',
 'neverhood',
 'saygus',
 'analfaces',
 'aquariumquestions',
 'saddestof',
 '100pushups',
 'malegrouptherapy',
 'drummers',
 'highered',
 'thisismydeathnow',
 'ozgamecollecting',
 'apopalypse',
 'pm_your__problems',
 'askveddit',
 'thirtyonedaychallenge',
 'shinyguild',
 'advancedproduction',
 'concordiaofthesnows',
 'rainmeter',
 'adultflashgames',
 'askaparapsychologist',
 'sillydrawingscontest',
 'thebayesianconspiracy',
 'bestofcraigslist',
 'malelifestyle',
 'achappyhome',
 'patents',
 'fut',
 'dicks',
 'lakka',
 'pbsspacetime',
 'otheroffmychest',
 'episodetalk',
 'recoveringliberals',
 'growthhacking',
 'unexpectedsips',
 'pokemongomunich',
 'uniqueke

In [24]:
subreddits = pd.read_csv('./input/web-redditEmbeddings-subreddits.csv', header = None) #Subreddit embeddings

body = pd.read_csv('./input/soc-redditHyperlinks-body.tsv', delimiter = '\t')
title = pd.read_csv('./input/soc-redditHyperlinks-title.tsv', delimiter = '\t')
subreddits.columns= ['name'] + ['e' + str(i) for i in subreddits.loc[:, 1:].columns]


temp_normalised_embeddings = np.divide(subreddits.loc[:, 'e1':], np.array(np.sqrt(np.square(subreddits.loc[:, 'e1':]).sum(axis=1))).reshape((-1, 1)),)
to_remove = temp_normalised_embeddings[temp_normalised_embeddings.isnull().any(axis = 1)].index.values
subreddits = subreddits.drop(to_remove, axis = 0).reset_index(drop = True)

temp_normalised_embeddings = temp_normalised_embeddings.drop(labels = to_remove, axis = 0).reset_index(drop = True)
embeddings = temp_normalised_embeddings.to_dict(orient = 'index')

'''with open('./input/embeddings.pickle', 'wb') as handle:
  pickle.dump(embeddings, handle, pickle.HIGHEST_PROTOCOL)'''

combined = pd.concat([title, body], axis = 0).reset_index(drop = True)

source = combined.SOURCE_SUBREDDIT.unique()
target = combined.TARGET_SUBREDDIT.unique()

left = combined[['POST_ID', 'TIMESTAMP', 'LINK_SENTIMENT', 'PROPERTIES']]
combined.drop(labels = ['POST_ID', 'TIMESTAMP', 'LINK_SENTIMENT', 'PROPERTIES'], axis = 1, inplace = True)

temp_mapper = subreddits['name'].to_dict()

mapper = {v:k for k,v in temp_mapper.items()}

combined.SOURCE_SUBREDDIT = combined.SOURCE_SUBREDDIT.map(mapper)
combined.TARGET_SUBREDDIT = combined.TARGET_SUBREDDIT.map(mapper)

combined = combined.dropna(axis = 0, how = 'any').reset_index(drop = True).astype(int)

import random
to_keep = random.sample(range(len(combined)), k = int(round(len(combined) * 0.1)))
combined = combined.loc[to_keep, :].reset_index(drop = True)

combined.to_csv('./input/graph.txt', header = None, index=None, sep=' ')

In [25]:
combined.shape

(76884, 2)

In [26]:
G = nx.read_edgelist('./input/graph.txt', nodetype=int, create_using=nx.DiGraph())
for edge in G.edges():
    G[edge[0]][edge[1]]['weight'] = 1

G = G.to_undirected()
'''for se in nx.nodes_with_selfloops(G):
    G.remove_edge(se, se)'''

print("Read graph, nodes: %d, edges: %d" % (G.number_of_nodes(), G.number_of_edges()))

Read graph, nodes: 15164, edges: 46694


In [62]:
S = [G.subgraph(c).copy() for c in nx.connected_components(G)]
S

In [55]:
[G.subgraph(c) for c in nx.connected_components(G)]

In [ ]:
n_edges = G.number_of_edges()
n_nodes = G.number_of_nodes()
npos = int(0.5 * n_edges)
nneg = int(0.5 * n_edges)
n_neighbors = [len(list(G.neighbors(v))) for v in list(G.nodes())]

In [ ]:
n_non_edges = n_nodes - 1 - np.array(n_neighbors)
n_non_edges

In [ ]:
non_edges = [e for e in nx.non_edges(G)]
print("Finding %d of %d non-edges" % (nneg, len(non_edges)))

In [ ]:
len(non_edges)

In [17]:
_rnd = np.random.RandomState(seed=1)
rnd_inx = _rnd.choice(len(non_edges), nneg, replace=False)
neg_edge_list = [non_edges[ii] for ii in rnd_inx]

In [18]:
list(G.edges())

NameError: name 'G' is not defined

In [205]:
edges = list(G.edges())
pos_edge_list = []
n_count = 0
n_ignored_count = 0
rnd_inx = _rnd.permutation(n_edges)
for eii in rnd_inx:
    edge = edges[eii]

    # Remove edge from graph
    data = G[edge[0]][edge[1]]
    G.remove_edge(*edge)

    # Check if graph is still connected
    #TODO: We shouldn't be using a private function for bfs
    reachable_from_v1 = nx.connected._plain_bfs(G, edge[0])
    if edge[1] not in reachable_from_v1:
        G.add_edge(*edge, **data)
        n_ignored_count += 1
    else:
        pos_edge_list.append(edge)
        print("Found: %d    " % (n_count), end="\r")
        n_count += 1

    # Exit if we've found npos nodes or we have gone through the whole list
    if n_count >= npos:
        break

if len(pos_edge_list) < npos:
    raise RuntimeWarning("Only %d positive edges found." % (n_count))

_pos_edge_list = pos_edge_list
_neg_edge_list = neg_edge_list

In [209]:
def train_embeddings(p, q, dimensions, num_walks, walk_length, window_size):
        """
        Calculate nodde embedding with specified parameters
        :param p:
        :param q:
        :param dimensions:
        :param num_walks:
        :param walk_length:
        :param window_size:
        :return:
        """
        node2vec.preprocess_transition_probs()
        walks = node2vec.simulate_walks(num_walks, walk_length)
        learn_embeddings(
            walks, dimensions, window_size
        )
        return embeddings
    
def learn_embeddings(walks, dimensions, window_size=10, niter=5):
        '''
        Learn embeddings by optimizing the Skipgram objective using SGD.
        '''
        # TODO: Python27 only
        walks = [map(str, walk) for walk in walks]
        model = Word2Vec(walks,
                         size=dimensions,
                         window=window_size,
                         min_count=0,
                         sg=1,
                         iter=niter)
        wvecs = model.wv
        return wvecs